In [1]:
! pip install gliner
! pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.0 MB/s eta 0:00:00


In [2]:
# download data
! wget https://huggingface.co/datasets/urchade/synthetic-pii-ner-mistral-v1/resolve/main/data.json

--2025-01-09 09:16:54--  https://huggingface.co/datasets/urchade/synthetic-pii-ner-mistral-v1/resolve/main/data.json
Resolving huggingface.co (huggingface.co)... 13.35.202.121, 13.35.202.34, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.121|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/57/a8/57a8ec4f4ec288845876da262e329709a3abab15f5604211687806ff1c31f16f/78680af45ca6b1175c502db2eec441933de7383f440fc0f0bae46a72be13c0c7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.json%3B+filename%3D%22data.json%22%3B&response-content-type=application%2Fjson&Expires=1736673414&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjY3MzQxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU3L2E4LzU3YThlYzRmNGVjMjg4ODQ1ODc2ZGEyNjJlMzI5NzA5YTNhYmFiMTVmNTYwNDIxMTY4NzgwNmZmMWMzMWYxNmYvNzg2ODBhZjQ1Y2E2YjExNzVjNTAyZGIyZWVjNDQxOTMzZGU3MzgzZjQ0MGZ

In [3]:
import json
import random

In [4]:
train_path = "data.json"

with open(train_path, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))

random.shuffle(data)
print('Dataset is shuffled...')

train_dataset = data[:int(len(data)*0.9)]
test_dataset = data[int(len(data)*0.9):]

print('Dataset is splitted...')

Dataset size: 19635
Dataset is shuffled...
Dataset is splitted...


In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

In [6]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = GLiNER.from_pretrained("urchade/gliner_small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/611M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
# use it for better performance, it mimics original implementation but it's less memory efficient
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [8]:
device

device(type='cuda', index=0)

In [9]:
# Optional: compile model for faster training
model.to(device)
print("done")

done


In [10]:
# calculate number of epochs
num_steps = 500
batch_size = 8
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 100,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-8fc1f212d808>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Step,Training Loss,Validation Loss
500,140.383400,87.537567
1000,92.862400,73.570969
1500,86.301500,69.578079
2000,80.092900,68.381248


/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 404 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 503 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


TrainOutput(global_step=2209, training_loss=97.64271472880829, metrics={'train_runtime': 1912.6162, 'train_samples_per_second': 9.239, 'train_steps_per_second': 1.155, 'total_flos': 0.0, 'train_loss': 97.64271472880829, 'epoch': 1.0})

In [13]:
trained_model = GLiNER.from_pretrained("models/checkpoint-1400", load_tokenizer=True, local_files_only=True)

config.json not found in /content/models/checkpoint-1400


In [14]:
text = """
Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored over 850 official senior career goals for club and country, making him the top goalscorer of all time.
"""

# Labels for entity prediction
labels = ["Person", "Award"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Cristiano Ronaldo dos Santos Aveiro => Person
Ballon d'Or => Award
UEFA Men's Player of the Year Awards => Award
European Golden Shoes => Award


In [15]:
def mask_pii(text, model, labels=None, threshold=0.5, mask_char='*'):
    """
    Mask PII entities in text using a trained GLiNER model.

    Args:
        text (str): Input text to mask
        model: Trained GLiNER model
        labels (list): List of entity labels to mask. If None, masks all detected entities
        threshold (float): Confidence threshold for entity detection
        mask_char (str): Character to use for masking

    Returns:
        tuple: (masked_text, masked_entities)
            - masked_text (str): Text with PII entities masked
            - masked_entities (list): List of dictionaries containing the original entities and their masks
    """
    # Get predictions from model
    entities = model.predict_entities(text, labels, threshold=threshold)

    # Sort entities by start position in reverse order
    # This prevents issues with changing string indices when we mask multiple entities
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    # Create a mutable version of the text
    masked_text = text
    masked_entities = []

    for entity in entities:
        # Create mask of same length as entity
        mask = mask_char * len(entity['text'])

        # Replace entity with mask in text
        masked_text = (
            masked_text[:entity['start']] +
            mask +
            masked_text[entity['end']:]
        )

        # Store masked entity info
        masked_entities.append({
            'original': entity['text'],
            'mask': mask,
            'label': entity['label'],
            'confidence': entity['confidence'],
            'position': (entity['start'], entity['end'])
        })

    return masked_text, masked_entities

In [16]:
def print_masked_results(text, model, labels=None):
    """Helper function to demonstrate masking results"""
    masked_text, masked_entities = mask_pii(text, model, labels)

    print("Original Text:")
    print(text)
    print("\nMasked Text:")
    print(masked_text)
    print("\nMasked Entities:")
    for entity in masked_entities:
        print(f"{entity['original']} => {entity['mask']} ({entity['label']}, confidence: {entity['confidence']:.3f})")


In [19]:
def mask_pii(text, model, labels=None, threshold=0.5, mask_char='*'):
    """
    Mask PII entities in text using a trained GLiNER model.

    Args:
        text (str): Input text to mask
        model: Trained GLiNER model
        labels (list): List of entity labels to mask. If None, masks all detected entities
        threshold (float): Confidence threshold for entity detection
        mask_char (str): Character to use for masking

    Returns:
        tuple: (masked_text, masked_entities)
            - masked_text (str): Text with PII entities masked
            - masked_entities (list): List of dictionaries containing the original entities and their masks
    """
    # Get predictions from model
    entities = model.predict_entities(text, labels, threshold=threshold)

    # Sort entities by start position in reverse order
    # This prevents issues with changing string indices when we mask multiple entities
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    # Create a mutable version of the text
    masked_text = text
    masked_entities = []

    for entity in entities:
        # Create mask of same length as entity
        mask = mask_char * len(entity['text'])

        # Replace entity with mask in text
        masked_text = (
            masked_text[:entity['start']] +
            mask +
            masked_text[entity['end']:]
        )

        # Store masked entity info
        masked_entity = {
            'original': entity['text'],
            'mask': mask,
            'label': entity['label'],
            'position': (entity['start'], entity['end'])
        }

        # Add confidence if available
        if 'confidence' in entity:
            masked_entity['confidence'] = entity['confidence']

        masked_entities.append(masked_entity)

    return masked_text, masked_entities

# Example usage with your trained model
def print_masked_results(text, model, labels=None):
    """Helper function to demonstrate masking results"""
    masked_text, masked_entities = mask_pii(text, model, labels)

    print("Original Text:")
    print(text)
    print("\nMasked Text:")
    print(masked_text)
    print("\nMasked Entities:")
    for entity in masked_entities:
        entity_info = f"{entity['original']} => {entity['mask']} ({entity['label']}"
        if 'confidence' in entity:
            entity_info += f", confidence: {entity['confidence']:.3f}"
        entity_info += ")"
        print(entity_info)

# Your existing labels
labels = ["Person", "Award"]

# Use the masking function
print_masked_results(text, trained_model, labels)

Original Text:

Cristiano Ronaldo dos Santos Aveiro (Portuguese pronunciation: [kɾiʃˈtjɐnu ʁɔˈnaldu]; born 5 February 1985) is a Portuguese professional footballer who plays as a forward for and captains both Saudi Pro League club Al Nassr and the Portugal national team. Widely regarded as one of the greatest players of all time, Ronaldo has won five Ballon d'Or awards,[note 3] a record three UEFA Men's Player of the Year Awards, and four European Golden Shoes, the most by a European player. He has won 33 trophies in his career, including seven league titles, five UEFA Champions Leagues, the UEFA European Championship and the UEFA Nations League. Ronaldo holds the records for most appearances (183), goals (140) and assists (42) in the Champions League, goals in the European Championship (14), international goals (128) and international appearances (205). He is one of the few players to have made over 1,200 professional career appearances, the most by an outfield player, and has scored 

In [21]:
def mask_pii(text, model, threshold=0.5):
    """
    Mask PII entities in text using a trained GLiNER model with comprehensive PII patterns.

    Args:
        text (str): Input text to mask
        model: Trained GLiNER model
        threshold (float): Confidence threshold for entity detection

    Returns:
        tuple: (masked_text, masked_entities)
    """
    # Comprehensive list of PII entity types
    pii_labels = [
        "Person",                 # Names
        "Organization",           # Company/org names
        "Location",              # Addresses, cities, countries
        "Date",                  # Dates and timestamps
        "Email",                 # Email addresses
        "PhoneNumber",           # Phone numbers
        "IDNumber",              # SSN, passport numbers, etc.
        "CreditCard",            # Credit card numbers
        "IPAddress",             # IP addresses
        "URL",                   # Web URLs
        "AccountNumber",         # Bank account numbers
        "License",               # License numbers
        "VehicleIdentifier",     # VIN numbers
        "MedicalRecord",         # Medical record numbers
        "Credential",            # Usernames, passwords
        "FinancialInfo",         # Financial account info
        "Age",                   # Age information
        "Gender",                # Gender information
        "Nationality",           # Nationality/citizenship info
        "Education",             # Educational information
        "Occupation",            # Job titles and workplace info
        "BiometricData",         # Biometric identifiers
        "SocialMedia",           # Social media handles
        "DeviceID",              # Device identifiers
        "Award",                 # Awards and honors
    ]

    # Custom mask characters for different types of PII
    mask_patterns = {
        "Person": "[NAME]",
        "Organization": "[ORG]",
        "Location": "[LOC]",
        "Date": "[DATE]",
        "Email": "[EMAIL]",
        "PhoneNumber": "[PHONE]",
        "IDNumber": "[ID]",
        "CreditCard": "[CC]",
        "IPAddress": "[IP]",
        "URL": "[URL]",
        "AccountNumber": "[ACC]",
        "License": "[LIC]",
        "VehicleIdentifier": "[VIN]",
        "MedicalRecord": "[MED]",
        "Credential": "[CRED]",
        "FinancialInfo": "[FIN]",
        "Age": "[AGE]",
        "Gender": "[GENDER]",
        "Nationality": "[NAT]",
        "Education": "[EDU]",
        "Occupation": "[OCC]",
        "BiometricData": "[BIO]",
        "SocialMedia": "[SOCIAL]",
        "DeviceID": "[DEVICE]",
        "Award": "[AWARD]"
    }

    # Get predictions from model
    entities = model.predict_entities(text, pii_labels, threshold=threshold)

    # Sort entities by start position in reverse order
    entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    masked_text = text
    masked_entities = []

    for entity in entities:
        # Get appropriate mask for this entity type
        mask = mask_patterns.get(entity['label'], f"[{entity['label']}]")

        # Replace entity with mask in text
        masked_text = (
            masked_text[:entity['start']] +
            mask +
            masked_text[entity['end']:]
        )

        # Store masked entity info
        masked_entity = {
            'original': entity['text'],
            'mask': mask,
            'label': entity['label'],
            'position': (entity['start'], entity['end'])
        }

        # Add confidence if available
        if 'confidence' in entity:
            masked_entity['confidence'] = entity['confidence']

        masked_entities.append(masked_entity)

    return masked_text, masked_entities


In [22]:
def print_masked_results(text, model):
    """Helper function to demonstrate masking results with statistics"""
    masked_text, masked_entities = mask_pii(text, model)

    print("Original Text:")
    print("-" * 80)
    print(text)
    print("\nMasked Text:")
    print("-" * 80)
    print(masked_text)
    print("\nDetected PII Entities:")
    print("-" * 80)

    # Group entities by type for better reporting
    entity_types = {}
    for entity in masked_entities:
        if entity['label'] not in entity_types:
            entity_types[entity['label']] = []
        entity_types[entity['label']].append(entity)

    # Print summary by entity type
    for label, entities in entity_types.items():
        print(f"\n{label} ({len(entities)} found):")
        for entity in entities:
            entity_info = f"  • {entity['original']} => {entity['mask']}"
            if 'confidence' in entity:
                entity_info += f" (confidence: {entity['confidence']:.3f})"
            print(entity_info)

    print("\nSummary Statistics:")
    print("-" * 80)
    print(f"Total PII entities found: {len(masked_entities)}")
    print(f"Unique PII types found: {len(entity_types)}")
    print("Distribution:")
    for label, entities in entity_types.items():
        print(f"  • {label}: {len(entities)}")

In [23]:

sample_text = """
John Smith works as a Senior Software Engineer at TechCorp Industries.
He can be reached at john.smith@techcorp.com or +1 (555) 123-4567.
His employee ID is TC-123456, and he uses the username 'jsmith' on the company network.
Born on March 15, 1985, in New York City, he holds a Master's degree in Computer Science from MIT.
John's projects earned him the Outstanding Innovation Award in 2023.
For secure access, use IP address 192.168.1.100 and device ID TCT-89X-2024.
Social Security Number: 123-45-6789
Credit Card: 4111-1111-1111-1111
"""



In [24]:
print_masked_results(sample_text, trained_model)

Original Text:
--------------------------------------------------------------------------------

John Smith works as a Senior Software Engineer at TechCorp Industries. 
He can be reached at john.smith@techcorp.com or +1 (555) 123-4567. 
His employee ID is TC-123456, and he uses the username 'jsmith' on the company network.
Born on March 15, 1985, in New York City, he holds a Master's degree in Computer Science from MIT.
John's projects earned him the Outstanding Innovation Award in 2023.
For secure access, use IP address 192.168.1.100 and device ID TCT-89X-2024.
Social Security Number: 123-45-6789
Credit Card: 4111-1111-1111-1111


Masked Text:
--------------------------------------------------------------------------------

[NAME] works as a [OCC] at [ORG]. 
He can be reached at [EMAIL] or [PHONE]. 
His employee ID is [DEVICE], and he uses the username 'jsmith' on the company network.
Born on [DATE], in [LOC], he holds a Master's degree in Computer Science from [ORG].
John's projects 